### 用正则匹配规则search

In [1]:
import regex as re

example1 = ['.{0,10}(王先生|你).{0,20}(外观|动力).{0,30} ', '.{0,25}(没关系|不用担心|别担心).{0,35}(试驾|是假|事假).{0,25}']

query1 = '这款车自从今年面市以来，几乎每天都有不少客户来试驾，您看我们的预约表每天都安排的满满的。'

In [43]:
lgf = '.{0,35}(试驾|预约|安排|客户).{0,50}(试驾|是假|事假).{0,15}'

def read_lgf(lfg):
    items = [item for item in lgf.strip().split('.') if item]
    ans = []
    for item in items:
        result = find_item(item)
        ans.append(result)
    return ans
         
def find_item(item):
    pos, endpos = [eval(i) for i in item[item.find('{') + 1: item.find('}')].split(',') if i.isdigit()]
    keyword = ''
    pattern=None
    if item.find('(') != -1:
        keyword = item[item.find('(') + 1: item.find(')')]
    if keyword:
        pattern = re.compile(keyword)
    return {'pos': pos, 'endpos': endpos, 'keyword': keyword, 'pattern': pattern}

response = read_lgf(lgf)
print('response: ', response)

response:  [{'pos': 0, 'endpos': 35, 'keyword': '试驾|预约|安排|客户', 'pattern': regex.Regex('试驾|预约|安排|客户', flags=regex.V0)}, {'pos': 0, 'endpos': 50, 'keyword': '试驾|是假|事假', 'pattern': regex.Regex('试驾|是假|事假', flags=regex.V0)}, {'pos': 0, 'endpos': 15, 'keyword': '', 'pattern': None}]


In [45]:
def match(text, patterns):
    '''
    1. 在 pattern 要求在规定的 pos 和 endpos 位置进行搜索，同时记录最右边的位置 可能有多个
    2. pattern的最后一个距离需求中得到 关键词 的距离要求
    3. 获得 pattern 内部关键词之间距离范围
    4. 比较距离范围是否在 3 的要求中
    '''
    ans = []
    span = None
    for p in patterns:
        if p['keyword'] and p['pattern']:
            temp = []
            pattern = p['pattern']
            for item in pattern.finditer(text, pos=p['pos'], endpos=p['endpos']):
                temp.append(item.span())
            if len(temp) == 0:
                return False
            # 如果 temp 中有多个结果，则以 first 结果为准  
            ans.append(temp[0])
        else:
            if p['endpos'] > 0:
                span = [p['pos'], p['endpos']]
    # 判断是否满足 keyword 的间距是否满足要求
    ## 情况一： 对 区间 无要求
    if not span:
        return True
    ## 情况二： 对 区间 有要求，则进行判断
    endpos = ans[0][1]
    for pos in ans[1:]:
        gap = pos[0] - endpos
        if gap < span[0] or gap > span[1]:
            return False
        endpos = pos[1]
    return True
print('query1: ', query1)
print('pattern: ', response)
ans = match(query1, response)
print('match: ', ans)


query1:  这款车自从今年面市以来，几乎每天都有不少客户来试驾，您看我们的预约表每天都安排的满满的。
pattern:  [{'pos': 0, 'endpos': 35, 'keyword': '试驾|预约|安排|客户', 'pattern': regex.Regex('试驾|预约|安排|客户', flags=regex.V0)}, {'pos': 0, 'endpos': 50, 'keyword': '试驾|是假|事假', 'pattern': regex.Regex('试驾|是假|事假', flags=regex.V0)}, {'pos': 0, 'endpos': 15, 'keyword': '', 'pattern': None}]
ans:  [(20, 22), (23, 25)] span:  [0, 15]
match:  True


In [36]:
ans[0].spans()

[(20, 22)]